In [1]:
!brew install wget

==> Auto-updating Homebrew...
Adjust how often this is run with HOMEBREW_AUTO_UPDATE_SECS or disable with
HOMEBREW_NO_AUTO_UPDATE. Hide these hints with HOMEBREW_NO_ENV_HINTS (see `man brew`).
==> Auto-updated Homebrew!
Updated 2 taps (homebrew/services and homebrew/core).
==> New Formulae
haproxy@2.8         openfa              pedump              span-lite

You have 5 outdated formulae installed.

==> Downloading https://ghcr.io/v2/homebrew/core/wget/manifests/1.24.5
######################################################################### 100.0%
==> Fetching wget
==> Downloading https://ghcr.io/v2/homebrew/core/wget/blobs/sha256:9befdad158e59
######################################################################### 100.0%
==> Pouring wget--1.24.5.arm64_sonoma.bottle.tar.gz
🍺  /opt/homebrew/Cellar/wget/1.24.5: 92 files, 4.5MB
==> Running `brew cleanup wget`...
Disable this behaviour by setting HOMEBREW_NO_INSTALL_CLEANUP.
Hide these hints with HOMEBREW_NO_ENV_HINTS (see `man brew`).


In [3]:
# Download the tiny Shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt --no-check-certificate

--2024-06-01 17:04:36--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  5.29MB/s    in 0.2s    

2024-06-01 17:04:37 (5.29 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [35]:
# First, we read data from the text file into a Python's variable
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()
    

In [36]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [37]:
# Get the vocabulary out of the data
chars = sorted(list(set(text))) # All the unique characters that occur in this text
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


**Vocabulary of the model** is: "\\n!$&\',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz65"

In [38]:
"""Create tokenizer"""

# Create a mapping from characters to intergers and vice versa
stoi = {ch:i for i,ch in enumerate(chars)} # A mapping in Python is in the form of a dictionary.
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Test encoder and decoder
print(encode("Hello there!"))
print(decode(encode("Hello there!")))

[20, 43, 50, 50, 53, 1, 58, 46, 43, 56, 43, 2]
Hello there!


### **What is the trade-off between the length of the sequence of integers - result of the encoder - and the size of the vocabulary?**
**There is a trade-off between the length of the sequence of integers - result of the encoder - and the size of the vocabulary.** Encoder having larger-sized vocab usually results in shorter output sequence of integers, and vice versa.
Other tokenizer for larger models: OpenAI's tiktoken and Google's sentencepiece, etc.

In [8]:
!pip3 install torch torchvision torchaudio

In [6]:
# Encode the entire text dataset and store it into a torch.Tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [39]:
"""Create train-test set"""
n = int(0.9*len(data))
train_data = data[:n] # Let 90% of the data to be the train data.
val_data = data[n:]

### **How do we feed the data into the Transformer?**
We don't ever feed the whole data into the Transformer when it comes to training. Instead, we sample fix-sized chunks of data and feed into the Transformer.
Each chunk/block of data contain multiple examples for training. For instance, a block consisting of 8 integers like [18, 47, 56, 57, 58,  1, 15, 47, 58] will be considered as 8 examples: [18], [18,47], [18,47,56], and so on while the Transformer is learning.

In [42]:
block_size = 8 # Maximum context length
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [43]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f'when input is {context} the target: {target}')

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


### **What is a batch?**
Actually, when we feed chunks of text (actually integers) into the Transformer, we feed a **batch** consisting of multiple chunks of text. So a batch is nothing but a tensor consisting of multiple tensors. This helps utilize the parallel computing power of the GPU.

In [44]:
torch.manual_seed(1337) # For reproducability
batch_size = 4 # How many independent sequences/chunks/blocks will we process in parallel? = B
block_size = 8 # What is the maximum context length for predictions? = T

def get_batch(split):
  # Generate a small batch of data of inputs x and targets y
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,)) # Generate 4 indices that are smaller than len(data) - block_size
                                                            # https://pytorch.org/docs/stable/generated/torch.randint.html
  x = torch.stack([data[i:i+block_size] for i in ix]) # Pull out chunks from data and stack 'em up as a 4x8 tensor
                                                      # https://pytorch.org/docs/stable/generated/torch.stack.html
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])

  return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # Batch dimension
  for t in range(block_size): # Time dimension
    context = xb[b, :t+1]
    target = yb[b,t]
    print(f'when input is {context.tolist()} the target: {target}')

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

In [45]:
print(xb) # Our input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


### **What is n-gram language model?**
**Bigram** or **2-gram language models** are models (a.k.a systems) that predict the next word based on the only preceeding word.

In [46]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
  # https://pytorch.org/docs/stable/generated/torch.nn.Module.html

  def __init__(self, vocab_size):
    super().__init__()
    # Each token directly reads off the logits for the next token from a lookup table
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # Init the lookup table
    # Each of the embedding is a vector of the size 65.

  def forward(self, idx, targets=None):

    # idx and targets are both (B,T) tensor of integers
    logits = self.token_embedding_table(idx)  # (B,T,C) with B = batch_size = 4
                                              # T = block_size = 8
                                              # C = vocab_size = 65
    # Debug
    # print("logits before\n", logits, "\n")
    # print(logits.shape)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)  # B*T is the total number of blocks
                                    # in all batches
      targets = targets.view(-1)    # or targets = targets.view(B*T)  
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):
      # get the predictions
      logits, loss = self(idx)          # = self.forward(idx)
      # focus only on the last time step
      logits = logits[:, -1, :]         # For all batches, get their last block
                                        # take all elements of those blocks
      # apply softmax to get probabilites
      probs = F.softmax(logits, dim=-1) # Transform all those blocks above
                                        # into probability distribution
      # sample from the distribution
      idx_next = torch.multinomial(probs, num_samples=1)
      # append sampled index to the running sequence
      idx = torch.cat((idx, idx_next), dim=1)   # (B, T+1)
                                                # idx_next is an index,
                                                # which will be concatenated into each and every batch in idx.

      # # Debug
      # print("logits after - the last time step\n", logits, "\n")
      # print(logits.shape)
      # print("probs\n", probs, "\n")
      # print("idx_next\n", idx_next, "\n")

    return idx

m = BigramLanguageModel(vocab_size)

# print(logits.shape)
# print(loss)

idx = torch.zeros((1, 1), dtype=torch.long) # Input token: a single whitespace

print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [47]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [48]:
batch_size = 32
for steps in range(10000):

  # sample a batch of data
  xb, yb = get_batch('train')

  # evaluate the loss
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

2.382369041442871


In [49]:
result = m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0]
print(result)
print(decode(result.tolist()))

tensor([ 0, 50, 57, 53,  1, 40, 56,  8,  1, 39, 60, 43,  1, 39, 60, 47, 39, 57,
        59, 56, 44,  1, 51, 63,  6,  1, 63, 62, 25, 28, 38, 21,  1, 47, 60, 43,
        43,  1, 47, 59, 43, 42, 56, 42,  1, 61, 46, 39, 56,  1, 49, 57, 58, 46,
         1, 63,  1, 46,  1, 40, 53, 56, 39,  1, 57,  1, 40, 43,  1, 46, 43, 57,
        43,  6,  1, 61, 53, 61, 43, 43, 43, 11,  1, 58, 46, 43,  2,  1, 23, 21,
         1,  5, 42, 43,  6,  1, 59, 50, 57, 43, 43, 41, 46, 43, 56, 42,  1, 42,
         1, 53,  1, 40, 50, 50, 50, 39, 52, 42, 53, 11, 24, 33, 15, 17, 27,  6,
         1, 53, 56, 39, 47, 52, 45, 53, 44, 53, 44,  1, 61, 47, 52,  2,  0, 30,
        21, 44, 39, 52, 57,  1, 54, 47, 41, 57, 54, 43, 57, 43, 56, 43, 56,  1,
        46, 43, 43,  1, 58, 46, 39,  6,  0, 32, 27, 18, 53, 52, 49, 12,  1, 51,
        43,  1, 39, 47, 52,  1, 41, 49, 52, 58, 53, 58, 63,  1, 42, 43, 42,  8,
         1, 40, 53,  5, 50, 50, 50, 50,  1, 57, 58,  1, 58, 39,  1, 42, 10,  0,
        17, 24, 21, 31,  1, 51, 43,  1, 

### **The mathematical trick in self-attention**

In [50]:
# Consider the following toy example
torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [51]:
# We want x[b,t] = mean_{i<=t} x[b,i]

# version 1
xbow = torch.zeros((B,T,C)) # "bow" stands for "bag-of-words".
for b in range(B):
  for t in range(T):
    xprev = x[b,:t+1] # (t,C)
    xbow[b,t] = torch.mean(xprev, 0)
xbow

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])

In [52]:
torch.tril(torch.ones(3, 3)) # Create a lower-triangular matrix

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [53]:
# version 2
wei = torch.tril(torch.ones(T, T))
wei = wei / torch.sum(wei, 1, keepdim=True)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [54]:
xbow2 = wei @ x # (B, T, T) @ (B, T, C)
xbow2

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])

In [55]:
torch.allclose(xbow[0], xbow2[0])

True

In [56]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill_(tril == 0, float("-inf"))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
# xbow3
# torch.allclose(xbow, xbow3)   # False?????? But xbow = xbow2
torch.allclose(xbow2, xbow3)  # True --> So xbow = xbow3

True

In [57]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True) # Normalize the triangular matrix
b = torch.randint(0,10,(3,2)).float()
c = a @ b

print("a = ", a, "\n")
print("b = ", b, "\n")
print("c = ", c, "\n")

a =  tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]]) 

b =  tensor([[2., 7.],
        [6., 4.],
        [6., 5.]]) 

c =  tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]]) 



### **Introduce "self-attention"**
Until now, the 3 version of **BoW** above only calculate the affinities between the current word and words in the past by calculating the average. In other words, we are considering every word in the past has the same relation to the current word. But this is not true in practice. In practice, there are words in the past are far more important to the current word than other ones.
In order to solve this problem of calculating "non-uniform" affinities, scientists have propose an approach called "**self-attention**". So, "self-attention" gathers data in the past words, but in a data-independent way.

In [58]:
# version 4: self-attention
torch.manual_seed(1337)
B,T,C = 4,8,32                # batch, time, channels
x = torch.randn((B,T,C))

# A single self-attention head
head_size = 16
key = nn.Linear(C, head_size, bias=False) # Recall: C = size of a vector embedding
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)                    # (B, T, 16)
q = query(x)                  # (B, T, 16)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ----> (B, T, T)
# .tranpose(-2, -1) means we transpose the last two dimensions.

tril = torch.tril(torch.ones(T,T))
# wei = torch.zeros((T,T))
wei = wei.masked_fill_(tril==0, float("-inf"))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
# out = wei @ x

out.shape

torch.Size([4, 8, 16])

### **Example of a wei matrix**
\begin{bmatrix}
-1.7629 & -1.3011 & 0.5652 & 2.1616 & -1.0674 & 1.9632 & 1.0765 & -0.4530 \\
-3.3334 & -1.6556 & 0.1040 & 3.3782 & -2.1825 & 1.0415 & -0.0557 & 0.2927 \\
-1.0226 & -1.2606 & 0.0762 & -0.3813 & -0.9843 & -1.4303 & 0.0749 & -0.9547 \\
0.7836 & -0.8014 & -0.3368 & -0.8496 & -0.5602 & -1.1701 & -1.2927 & -1.0260 \\
-1.2566 & 0.0187 & -0.7880 & -1.3204 & 2.0363 & 0.8638 & 0.3719 & 0.9258 \\
-0.3126 & 2.4152 & -0.1106 & -0.9931 & 3.3449 & -2.5229 & 1.4187 & 1.2196 \\
1.0876 & 1.9652 & -0.2621 & -0.3158 & 0.6091 & 1.2616 & -0.5484 & 0.8048 \\
-1.8044 & -0.4126 & -0.8306 & 0.5899 & -0.7987 & -0.5856 & 0.6433 & 0.6303 \\
\end{bmatrix}
  
**E.g:** In this batch of words, we can see that the 6th token (6th row) in this batch attends to the 5th token the most in its row, at 3.449.